In [1]:
import json
import math
import random
random.seed(42)
import typing

from arm import Arm
from production_line import Box, Lidar, Production_Line

In [3]:
with open('configs/production_line_config.json', 'r') as f:
    temp = f.readline()
    pl_config = json.loads(temp)

In [4]:
with open('configs/arm_config.json', 'r') as f:
    temp = f.readline()
    arm_config = json.loads(temp)

In [5]:
BOX_SIDE_LENGTH = pl_config['BOX_SIDE_LENGTH']
PRODUCTION_LINE_LENGTH = pl_config['PRODUCTION_LINE_LENGTH']
VELOCITY = pl_config['VELOCITY']

In [6]:
ARM_1_RANGE = arm_config['ARM_1_RANGE']
ARM_2_RANGE = arm_config['ARM_2_RANGE']
MAX_VARIABLE_ARM_LENGTH_UNIT = arm_config['MAX_VARIABLE_ARM_LENGTH_UNIT']
MAX_VARIABLE_ANGLE_UNIT = arm_config['MAX_VARIABLE_ANGLE_UNIT']
MIN_ARM_ZOOM_LENGTH = arm_config['MIN_ARM_ZOOM_LENGTH']
MIN_ANGLE_DEGREE = arm_config['MIN_ANGLE_DEGREE']
REWARD_DISCOUNT_FACTOR = arm_config['REWARD_DISCOUNT_FACTOR']
TERMINATION_CONDITION = arm_config['TERMINATION_CONDITION']

In [7]:
pl_1 = Production_Line(length=PRODUCTION_LINE_LENGTH, velocity=VELOCITY, box_side_length=BOX_SIDE_LENGTH)
pl_2 = Production_Line(length=PRODUCTION_LINE_LENGTH, velocity=VELOCITY, box_side_length=BOX_SIDE_LENGTH)

In [8]:
head_lidar = Lidar(place=BOX_SIDE_LENGTH)
tail_lidar = Lidar(place=PRODUCTION_LINE_LENGTH)

pl_1.add_lidar(head_lidar)
pl_1.add_lidar(tail_lidar)

In [9]:
head_lidar = Lidar(place=BOX_SIDE_LENGTH)
tail_lidar = Lidar(place=PRODUCTION_LINE_LENGTH)

pl_2.add_lidar(head_lidar)
pl_2.add_lidar(tail_lidar)

In [10]:
for _ in range(50):
    if pl_1.head_lidar_signal == False:
        box = Box(box_side_length=BOX_SIDE_LENGTH)
        pl_1.add_box(box)
    if pl_1.tail_lidar_signal == True and pl_2.head_lidar_signal == False:
        box = pl_1.remove_last_box()
        # arm
        # If the arm moves the box to the second destination in one unit of time, then pl_2 gets the box.
        pl_2.add_box(box)
    if pl_1.tail_lidar_signal == False:
        pl_1.shift()
    if pl_2.tail_lidar_signal == False:
        pl_2.shift()
    else:
        box = pl_2.remove_last_box()

In [11]:
arm = Arm(
    arm_1_range=ARM_1_RANGE, arm_2_range=ARM_2_RANGE, 
    min_arm_zoom_length=MIN_ARM_ZOOM_LENGTH, min_angle_degree=MIN_ANGLE_DEGREE,
    max_variable_arm_length_unit=MAX_VARIABLE_ARM_LENGTH_UNIT, max_variable_angle_unit=MAX_VARIABLE_ANGLE_UNIT,
    reward_discount_factor=REWARD_DISCOUNT_FACTOR, termination_condition=TERMINATION_CONDITION,
)

In [12]:
obs = arm.reset()

In [13]:
action = arm.generate_random_action()

In [14]:
obs, reward, done = arm.step(action=action)

In [15]:
obs

{'arm_1_length': 7.2,
 'arm_2_length': 4.0,
 'angle_1_degree': 98.69999999999999,
 'angle_2_degree': 79.8,
 'position': {'x': -0.5954809397805518,
  'y': 3.891485367606205,
  'z': 6.491661038721666},
 'targe': {'x': 2.88841080311338,
  'y': -2.3725673819262605,
  'z': -0.5704072475938506}}